# Bistable hinge - single hinge multiple shims

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import importlib 

from VariablesClass import VariablesClass
from SupervisorClass import  SupervisorClass
from StateClass import StateClass
from config import CFG
import colors, plot_func, funcs_geometry, funcs_physical, funcs_ML

In [ ]:
import config
importlib.reload(config)
from config import CFG

import StructureClass
importlib.reload(StructureClass)
from StructureClass import StructureClass

Strctr = StructureClass(CFG)

In [ ]:
import config
importlib.reload(config)
from config import CFG

import VariablesClass
importlib.reload(VariablesClass)
from VariablesClass import VariablesClass

Variabs = VariablesClass(CFG, Strctr)
Variabs.set_normalizations()

In [ ]:
importlib.reload(funcs_geometry)
importlib.reload(funcs_physical)
import SupervisorClass
importlib.reload(SupervisorClass)
from SupervisorClass import SupervisorClass

Sprvsr = SupervisorClass(CFG, Strctr)
Sprvsr.init_dataset(Strctr, Variabs)
Sprvsr.desired_tau(Variabs)

In [ ]:
import StateClass
importlib.reload(StateClass)
from StateClass import StateClass

State = StateClass(CFG, Strctr, Variabs, Sprvsr, buckle='from config')

## Loop

### Setup - measure a sweep over all angles using initial and desired configuration

In [ ]:
importlib.reload(funcs_geometry)
importlib.reload(funcs_physical)
importlib.reload(funcs_ML)

theta_full, tau_full_init = funcs_physical.measure_full_response(State.buckle, Variabs.theta_ss, Variabs.k_stiff,
                                                                 Variabs.k_soft)
if Sprvsr.desired_mode == 'specific_buckle':
    theta_full, tau_full_desired  = funcs_physical.measure_full_response(Sprvsr.desired_buckle, Variabs.theta_ss, 
                                                                         Variabs.k_stiff, Variabs.k_soft)
else:
    tau_full_desired = Sprvsr.desired_tau_func(theta_full)

In [ ]:
importlib.reload(plot_func)

plot_func.plot_response(theta_full, tau_full_init, tau_full_desired, theta_range=[theta_full[0], theta_full[-1]])

## Measure tau - single hinge

In [ ]:
loss_tot_init = np.mean((tau_full_desired-tau_full_init)**2/(Variabs.k_bar*Variabs.theta_bar**2)**2)

for t in range(Sprvsr.T):
    
    print('t=', t)

    ## Measure
    if Sprvsr.problem == 'tau':
        Sprvsr.set_theta(Variabs, t)
        State.calc_tau(Variabs, Sprvsr.theta, 0)
    elif Sprvsr.problem == 'Fy':
        Sprvsr.set_pos(Strctr, t)
        State.calc_Fy(Strctr, Variabs, Sprvsr.thetas)
    
    ## Loss
    Sprvsr.calc_loss(Variabs, State, t)

    ## Update
    Sprvsr.calc_input_update(State, Sprvsr, Variabs, t)
    ## Material evolution during Update
    State.evolve_material(Strctr, Variabs, Sprvsr, t)
    
    if t%Sprvsr.window_for_kill==Sprvsr.window_for_kill-1 and t>0:
        # print('the mean', Sprvsr.loss_MSE_in_t[t-Sprvsr.window_for_kill+1:t+1])
        if np.mean(Sprvsr.loss_MSE_in_t[t-Sprvsr.window_for_kill+1:t+1])<Sprvsr.eps:
            print('break at t=', t)
            break

            
print('final buckle=', State.buckle)
theta_full, tau_full_final  = funcs_physical.measure_full_response(State.buckle, Variabs.theta_ss, Variabs.k_stiff,
                                                                   Variabs.k_soft)
loss_tot_fin = np.mean((tau_full_desired-tau_full_final)**2/(Variabs.k_bar*Variabs.theta_bar**2)**2)

In [ ]:
if Sprvsr.problem == 'tau':
    importlib.reload(plot_func)

    plot_func.importants(State.buckle_in_t, Sprvsr.loss_MSE_in_t, Sprvsr.input_update_in_t, Sprvsr.desired_buckle,
                         window=16)

    # plot_func.plot_response(theta_full, tau_full_init, tau_full_desired, theta_range=[-90, 90])
    plot_func.plot_response(theta_full, tau_full_init, tau_full_desired, tau_full_final, theta_range=[-90, 90])

In [ ]:
print('loss_tot_init', loss_tot_init)
print('loss_tot_fin', loss_tot_fin)

In [ ]:
loss_full = tau_full_desired - tau_full_final
loss_norm_full = loss_full / (Variabs.k_bar * theta_full ** 2)
loss_MSE_full = loss_norm_full ** 2
plt.plot(theta_full, tau_full_desired, '.')
plt.plot(theta_full, loss_full, '.')
plt.plot(theta_full, 180*loss_norm_full, '.')
plt.plot(theta_full, 180*loss_MSE_full, '.')
plt.ylim([-40000, 40000])

In [ ]:
plt.plot(theta_full, tau_full_desired)

# Not in use

## Measure Fy - double hinge

In [ ]:
# if problem == 'Fy':
#     L=1
#     x = -0.28
#     y = 1.8
#     theta1, theta2 = theta_from_xy(x, y, -init_buckle[0])
#     print(np.degrees(theta1), np.degrees(theta2))
#     tau1 = tau_k(theta1, [1, 1], Variabs)[0]
#     tau2 = tau_k(theta2, [1, 1], Variabs)[1]
#     print(tau1, tau2)
#     Fy1 = Fy(theta1, theta2, tau1, tau2)
#     print(Fy1)

#     # Reachability check
#     reach_min, reach_max = 0, 2*L
#     r = np.hypot(x, y)
#     if r < reach_min - 1e-9 or r > reach_max + 1e-9:
#         print("WARNING: Target point is outside reachable workspace. The diagram uses clipped IK (c2 clipped).")

#     plot_arm(x, y, theta1, theta2)

## Fy = tau1*sin(theta1)